# Training Plan Progressions

In [82]:
from datetime import date, timedelta

In [95]:
def plan_week_range(start_date, num_weeks, start_week=0, step=1):
    # ('prog', 'rest', 'race')
    for wk in range(start_week, num_weeks, step):
        date = start_date + timedelta(weeks=wk)
        if wk == (num_weeks - 1):
            week_type = 'race'
        elif rest_week(wk, num_weeks):
            week_type = 'rest'
        else:
            week_type = 'prog'
        yield (wk, date, week_type)

In [96]:
def rest_week(week, plan_length):
    '''
    int -> boolean

    Determine if current week is a rest week.

    Plans work on a 4 week block, with every 4th week being an easier week.
    Runner has at least 2 weeks, and a maximum of 5 weeks before they get an
    easier week.  So if they were on a 6 week plan they would only have an
    easier week on race week.

    Returns True if rest week and False if progression week.
    '''
    build_up = plan_length % 4
    if week <= build_up and build_up < 3:
        return False
    elif (week - build_up + 1) % 4 == 0:
        return True
    else:
        return False

## Base Classes

In [97]:
class Exercise:

    def __init__(self, description, duration, intensity='Normal'):
        self.description = description
        self.duration = duration
        self.intensity = intensity
    
    def __repr__(self):
        '''
        Return a more human-readable representation
        '''
        return '{0}({1})'.format(self.description, self.duration)

    @staticmethod
    def mins_to_seconds_formatter(dur_in_mins):
        return "{}s".format(int(dur_in_mins * 60))

In [98]:
class WorkoutSet:

    def __init__(self, reps):
        self.reps = reps
        self.exercises = []
        self._duration = 0

    def __repr__(self):
        '''
        Return a more human-readable representation
        '''
        ex = ', '.join(str(exercise) for exercise in self.exercises)
        return '{0}x ({1})'.format(self.reps, ex)

    @property
    def duration(self):
        return self.reps * self._duration

    def add_exercise(self, exercise):
        self.exercises.append(exercise)
        self._duration += exercise.duration

In [99]:
class Workout:
    '''
    Represents a workout session
    '''

    formatting_dict = {
        'Event Day': {'color': '#001F3F',
                      'textColor': 'hsla(210, 100%, 75%, 1.0)'},
        'RunEasy': {'color': '#2ECC40',
                    'textColor': 'hsla(127, 63%, 15%, 1.0)'},
        'Intervals': {'color': '#FF4136',
                      'textColor': 'hsla(3, 100%, 25%, 1.0)'},
        'Hillsprint': {'color': '#FFDC00',
                       'textColor': 'hsla(52, 100%, 20%, 1.0)'},
        'Tempo': {'color': '#0074D9',
                  'textColor': 'hsla(208, 100%, 85%, 1.0)'}
    }

    def __init__(self, date, description):
        self.date = date
        self.description = description
        self.duration = 0
        self.workoutsets = []

    def __repr__(self):
        '''
        Return a more human-readable representation
        '''
        return '{0} - {1}'.format(self.date.strftime('%d %b %Y'),
                                  self.description)

    def __str__(self):
        '''
        Return a more human-readable representation
        '''
        # TODO: add if for EventDay
        ws = '\n'.join('{0}x {1}'.format(workoutset.reps,
                                         workoutset.exercises) for workoutset in self.workoutsets)
        return '{0}\n{1}'.format(self.description, ws)

    @property
    def color(self):
        return self.formatting_dict[self.description]['color']

    @property
    def textColor(self):
        return self.formatting_dict[self.description]['textColor']

    def add_workoutset(self, workoutset):
        self.workoutsets.append(workoutset)
        self.duration += workoutset.duration

In [100]:
class Progression:
    def __init__(self, start_date, length, start_week=0, step=1, func=None):
        self.start_date = start_date
        self.length = length
        self.start_week = start_week
        self.step = step
        self.sessions = []

        if func is not None:
            self.create = types.MethodType(func, self)

    def create(self):
        raise NotImplementedError

## Progression Functions

In [107]:
def create_runeasy(self, init_dur, prog_freq, rest_week_cut, max_dur):
    '''
    Return an easy running progression as a list of workouts
    '''

    dur = init_dur

    def week_cut_version(week_cut, dur):
        '''
        Return rest week duration based on wether cut is applied as an absolute
        or % value
        '''
        if isinstance(week_cut, str):
            return (float(week_cut.strip('%')) / 100) * dur
        else:
            return dur - week_cut

    for wk, date, wk_type in plan_week_range(self.start_date, self.length):
        if wk_type == 'prog':
            if (wk + 1) % prog_freq == 0 and dur < max_dur:
                dur += 5
            wk_dur = dur
        elif wk_type == 'rest':
            wk_dur = week_cut_version(rest_week_cut, dur)
        else:
            wk_dur = week_cut_version(race_week_cut, dur)

        w = Workout(date, 'RunEasy')
        ws = WorkoutSet(1)
        e = Exercise('Easy', wk_dur)
        ws.add_exercise(e)
        w.add_workoutset(ws)
        self.sessions.append(w)

In [108]:
import types

In [112]:
start_date = date(2017, 8, 25)
length = 8
start_week = 0
step = 1
func = create_runeasy

init_dur = 25
prog_freq = 3
rest_week_cut = 5
race_week_cut = 10
max_dur = 35

In [113]:
p = Progression(start_date, length, start_week, step, func)

In [114]:
p.create(init_dur, prog_freq, rest_week_cut, max_dur)

In [106]:
p.sessions

[25 Aug 2017 - RunEasy, 01 Sep 2017 - RunEasy, 08 Sep 2017 - RunEasy]